### Loading data

- PV/OP data
- SSD CSV
- Trip data

In [6]:
import pandas as pd

# Load OP PV data
df_related_KG_tags = pd.read_csv('DATA/03LIC1071_PropaneLoop_0426.csv')
# Get unique tag names from df_related_KG_tags
unique_tag_names = df_related_KG_tags['tagName'].unique()

op_pv_data_df = pd.read_parquet('/home/h604827/ControlActions/DATA/03LIC_1071_JAN_2026.parquet')
ssd_df = pd.read_excel("/home/h604827/ControlActions/DATA/SSD_1071_SSD_output_1071_7Jan2026.xlsx")

trip_data = pd.read_csv('/home/h604827/ControlActions/DATA/Final_List_Trip_Duration.csv')
op_pv_data_df.set_index('TimeStamp', inplace=True)
op_pv_data_df.sort_index(inplace=True)
op_pv_data_df.columns, trip_data.columns

(Index(['03LIC_1071.PV', '03LIC_1071.OP', '02FI_1000.PV', '03FIC_1085.OP',
        '03FIC_1085.PV', '03FIC_3415.OP', '03FIC_3415.PV', '03FIC_3435.PV',
        '03FI_1141A.PV', '03FI_1151.PV', '03FI_3418.PV', '03LIC_1016.OP',
        '03LIC_1016.PV', '03LIC_1085.OP', '03LIC_1085.PV', '03LIC_1094.OP',
        '03LIC_1094.PV', '03LIC_1097.OP', '03LIC_1097.PV', '03LIC_3178.OP',
        '03LIC_3178.PV', '03LI_3411.PV', '03PIC_1013.OP', '03PIC_1013.PV',
        '03PIC_1068.OP', '03PIC_1068.PV', '03PIC_1104.OP', '03PIC_1104.PV',
        '03PIC_3131.OP', '03PIC_3131.PV', '03PI_1141A.PV', '03PI_1495.PV',
        '03PI_1814.PV', '03TIC_1092.OP', '03TIC_1092.PV', '03TIC_1142.OP',
        '03TIC_1142.PV', '03TIC_1145.OP', '03TIC_1145.PV', '03TI_1015.PV',
        '03TI_1081.PV', '03TI_1421.PV', '03TI_1901.PV', 'AlarmStatus',
        'AlarmType'],
       dtype='object'),
 Index(['Stop Date', 'Start Date', 'Unit', 'SAP Location'], dtype='object'))

In [7]:
# Get the list of columns
cols = op_pv_data_df.columns

# Extract base tag names for OP and PV columns
op_tags = {col.replace('.OP', '') for col in cols if col.endswith('.OP')}
pv_tags = {col.replace('.PV', '') for col in cols if col.endswith('.PV')}

# Identify mismatches
op_without_pv = op_tags - pv_tags
pv_without_op = pv_tags - op_tags

print("Tags with OP but no PV:", op_without_pv)
print("Tags with PV but no OP:", pv_without_op)

Tags with OP but no PV: set()
Tags with PV but no OP: {'03FI_1151', '03TI_1421', '02FI_1000', '03TI_1081', '03PI_1814', '03TI_1015', '03PI_1141A', '03FI_1141A', '03FI_3418', '03PI_1495', '03LI_3411', '03TI_1901', '03FIC_3435'}


### Load events data and remove trip duration events

In [4]:
combined_pv_events_df = pd.read_csv("/home/h604827/ControlActions/DATA/df_df_events_1071_export.csv", low_memory=False)
combined_pv_events_df['VT_Start'] = pd.to_datetime(combined_pv_events_df['VT_Start'])
combined_pv_events_df = combined_pv_events_df.sort_values('VT_Start')

# Convert trip_data dates to datetime
trip_data['Stop Date'] = pd.to_datetime(trip_data['Stop Date'])
trip_data['Start Date'] = pd.to_datetime(trip_data['Start Date'])

# Use numpy for fast vectorized comparison
import numpy as np

event_times = combined_pv_events_df['VT_Start'].values
stop_dates = trip_data['Stop Date'].values
start_dates = trip_data['Start Date'].values

# Check if each event falls within any trip period using broadcasting
# Shape: (n_events, n_trips) -> then reduce with any()
events_in_trips = np.zeros(len(event_times), dtype=bool)

# Process in chunks to avoid memory issues with large arrays
chunk_size = 100000
for i in range(0, len(event_times), chunk_size):
    chunk = event_times[i:i+chunk_size]
    # Broadcasting: (chunk_size, 1) compared with (1, n_trips)
    in_range = (chunk[:, None] >= stop_dates) & (chunk[:, None] <= start_dates)
    events_in_trips[i:i+chunk_size] = in_range.any(axis=1)

# Filter out events that are within trip periods
print(f"Original events: {len(combined_pv_events_df)}")
print(f"Events during trips (to be removed): {events_in_trips.sum()}")
combined_pv_events_df = combined_pv_events_df[~events_in_trips]
print(f"Remaining events after filtering: {len(combined_pv_events_df)}")

combined_pv_events_df.shape

Original events: 1947510
Events during trips (to be removed): 346990
Remaining events after filtering: 1600520


(1600520, 29)

In [8]:
combined_pv_events_df.to_csv('/home/h604827/ControlActions/DATA/trip_filtered_events.csv', index=False)

In [9]:
# Remove trip duration data from op_pv_data_df

print("Filtering op_pv_data_df to remove trip periods...")
print(f"Original op_pv_data_df shape: {op_pv_data_df.shape}")

# Get timestamps from op_pv_data_df index
op_pv_timestamps = op_pv_data_df.index.values

# Use numpy for fast vectorized comparison
timestamps_in_trips = np.zeros(len(op_pv_timestamps), dtype=bool)

# Process in chunks to avoid memory issues with large arrays
chunk_size = 100000
for i in range(0, len(op_pv_timestamps), chunk_size):
    chunk = op_pv_timestamps[i:i+chunk_size]
    # Broadcasting: (chunk_size, 1) compared with (1, n_trips)
    in_range = (chunk[:, None] >= stop_dates) & (chunk[:, None] <= start_dates)
    timestamps_in_trips[i:i+chunk_size] = in_range.any(axis=1)

# Filter out timestamps that are within trip periods
print(f"Timestamps during trips (to be removed): {timestamps_in_trips.sum()}")
op_pv_data_df = op_pv_data_df[~timestamps_in_trips]
print(f"Remaining timestamps after filtering: {len(op_pv_data_df)}")
print(f"Filtered op_pv_data_df shape: {op_pv_data_df.shape}")

Filtering op_pv_data_df to remove trip periods...
Original op_pv_data_df shape: (1737586, 45)
Timestamps during trips (to be removed): 19547
Remaining timestamps after filtering: 1718039
Filtered op_pv_data_df shape: (1718039, 45)


In [10]:
op_pv_data_df.to_parquet('/home/h604827/ControlActions/DATA/03LIC_1071_JAN_2026_filtered.parquet')

### Operator actions for PIC_1013, 1071 and 1016 tag

In [5]:
combined_pv_events_df[combined_pv_events_df['ConditionName'] == 'CHANGE']['Source'].value_counts()

Source
03LIC_1034      40978
03FIC_3435      34252
03HIC_1151      21948
03HIC_3100      19412
03PIC_1013      15568
                ...  
03RH_0201           2
03HS_0151           2
03LT_1071           2
03ST_3199A          2
03KI_0049_LB        2
Name: count, Length: 245, dtype: int64

In [149]:
combined_pv_events_df[(combined_pv_events_df['Source'].isin(['03LIC_1071', '03LIC_1016', '03PIC_1013'])) & (combined_pv_events_df['ConditionName'] == 'CHANGE')]['Source'].value_counts()

Source
03PIC_1013    15568
03LIC_1071     3076
03LIC_1016     1860
Name: count, dtype: int64

In [150]:
combined_pv_events_df[(combined_pv_events_df['Source'] =='03LIC_1016') & (combined_pv_events_df['ConditionName'] == 'CHANGE')]['Description'].value_counts()

Description
OP      1096
SP       642
MODE     122
Name: count, dtype: int64

### Operators changed the mode to MAN for 1071 and 1016 alarms these many times

In [151]:
combined_pv_events_df[(combined_pv_events_df['Source'].isin(['03LIC_1071', '03LIC_1016'])) & (combined_pv_events_df['ConditionName'] == 'CHANGE') & (combined_pv_events_df['Description'] == 'MODE') & (combined_pv_events_df['Value'] == 'MAN')]

,Action,Actor,AreaName,AlarmLimit,Block,Category,ConditionName,Description,EventID,Flags,...,SourceParameter,Station,Time,TransactionID,Units,Value,VT_Start,H,TagID,AlarmStatus
546921,NaN,NaN,1E,NaN,NaN,7,CHANGE,MODE,31971423,NaN,...,NaN,NaN,132822814613603000,31971423,%,MAN,2021-11-25 06:37:41.360300,2021_11_25_08,NaN,NaN
6663,NaN,NaN,1E,NaN,NaN,7,CHANGE,MODE,31971423,NaN,...,NaN,NaN,132822814613603000,31971423,%,MAN,2021-11-25 06:37:41.360300,2021_11_25_08,03LIC_1016,NaN
7519,NaN,NaN,1F,NaN,NaN,7,CHANGE,MODE,31971445,NaN,...,NaN,NaN,132822814848530000,31971445,%,MAN,2021-11-25 06:38:04.853000,2021_11_25_08,03LIC_1071,NaN
568524,NaN,NaN,1F,NaN,NaN,7,CHANGE,MODE,31971445,NaN,...,NaN,NaN,132822814848530000,31971445,%,MAN,2021-11-25 06:38:04.853000,2021_11_25_08,NaN,NaN
8511,NaN,NaN,1F,NaN,NaN,7,CHANGE,MODE,33209244,NaN,...,NaN,NaN,132834813552533000,33209244,%,MAN,2021-12-09 03:55:55.253300,2021_12_09_05,03LIC_1071,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639544,NaN,NaN,1F,NaN,NaN,7,CHANGE,MODE,78594407,NaN,...,NaN,NaN,133911203810236000,78594407,%,MAN,2025-05-07 23:39:41.023600,2025_05_08_01,NaN,NaN
644160,NaN,NaN,1F,NaN,NaN,7,CHANGE,MODE,79410466,NaN,...,NaN,NaN,133949891544419000,79410466,%,MAN,2025-06-21 18:19:14.441900,2025_06_21_19,NaN,NaN
68375,NaN,NaN,1F,NaN,NaN,7,CHANGE,MODE,79410466,NaN,...,NaN,NaN,133949891544419000,79410466,%,MAN,2025-06-21 18:19:14.441900,2025_06_21_19,03LIC_1071,NaN
644171,NaN,NaN,1F,NaN,NaN,7,CHANGE,MODE,79410548,NaN,...,NaN,NaN,133949895420746000,79410548,%,MAN,2025-06-21 18:25:42.074600,2025_06_21_19,NaN,NaN


### Actions coming within [transition start, alarm end + 60 mins] for each alarm

In [152]:
# For each alarm episode, find the earliest Tag_First_Transition_Start_minutes
# and create a window from that timestamp to AlarmEnd + 60 minutes

import pandas as pd
import numpy as np

# Ensure datetime columns are properly parsed
ssd_df['Tag_First_Transition_Start_minutes'] = pd.to_datetime(ssd_df['Tag_First_Transition_Start_minutes'])
ssd_df['AlarmEnd_rounded_minutes'] = pd.to_datetime(ssd_df['AlarmEnd_rounded_minutes'])
ssd_df['AlarmStart_rounded_minutes'] = pd.to_datetime(ssd_df['AlarmStart_rounded_minutes'])

# Group by alarm episode (AlarmStart, AlarmEnd) and find the earliest transition start
alarm_windows = ssd_df.groupby(
    ['AlarmStart_rounded_minutes', 'AlarmEnd_rounded_minutes']
).agg(
    earliest_transition_start=('Tag_First_Transition_Start_minutes', 'min')
).reset_index()

# Create window end as AlarmEnd + 60 minutes
alarm_windows['window_start'] = alarm_windows['earliest_transition_start']
alarm_windows['window_end'] = alarm_windows['AlarmEnd_rounded_minutes'] + pd.Timedelta(minutes=60)

print(f"Total alarm windows: {len(alarm_windows)}")
print(f"\nFirst 5 alarm windows:")
print(alarm_windows[['AlarmStart_rounded_minutes', 'AlarmEnd_rounded_minutes', 
                      'window_start', 'window_end']].head())

# Now filter combined_pv_events_df for events within these windows
# Convert VT_Start to datetime if not already
combined_pv_events_df['VT_Start'] = pd.to_datetime(combined_pv_events_df['VT_Start'])

# Vectorized approach: check if each event falls within any window
event_times = combined_pv_events_df['VT_Start'].values
window_starts = alarm_windows['window_start'].values
window_ends = alarm_windows['window_end'].values

# Process in chunks to avoid memory issues
events_in_windows = np.zeros(len(event_times), dtype=bool)
alarm_episode_idx = np.full(len(event_times), -1, dtype=int)  # Track which alarm episode each event belongs to

chunk_size = 100000
for i in range(0, len(event_times), chunk_size):
    chunk = event_times[i:i+chunk_size]
    # Broadcasting: (chunk_size, 1) compared with (1, n_windows)
    in_range = (chunk[:, None] >= window_starts) & (chunk[:, None] <= window_ends)
    events_in_windows[i:i+chunk_size] = in_range.any(axis=1)
    # Get the first matching alarm episode index for each event
    for j, row_matches in enumerate(in_range):
        if row_matches.any():
            alarm_episode_idx[i+j] = np.where(row_matches)[0][0]

# Filter events that are within any alarm window
events_in_alarm_windows_df = combined_pv_events_df[events_in_windows].copy()
events_in_alarm_windows_df['AlarmEpisodeIndex'] = alarm_episode_idx[events_in_windows]

# Add alarm episode details
events_in_alarm_windows_df = events_in_alarm_windows_df.merge(
    alarm_windows[['AlarmStart_rounded_minutes', 'AlarmEnd_rounded_minutes', 
                   'window_start', 'window_end']].reset_index().rename(columns={'index': 'AlarmEpisodeIndex'}),
    on='AlarmEpisodeIndex',
    how='left'
)

events_in_alarm_windows_df = events_in_alarm_windows_df[(events_in_alarm_windows_df['Source'].isin(['03LIC_1071', '03LIC_1016', '03PIC_1013'])) & (events_in_alarm_windows_df['ConditionName'] == 'CHANGE')]
print(f"\n--- Events Filtering Summary ---")
print(f"Total events in combined_pv_events_df: {len(combined_pv_events_df)}")
print(f"Events within alarm windows: {len(events_in_alarm_windows_df)}")
print(f"Unique alarm episodes with events: {events_in_alarm_windows_df['AlarmEpisodeIndex'].nunique()}")
print(f"\nConditionName distribution in filtered events:")
print(events_in_alarm_windows_df['ConditionName'].value_counts().head(10))
print(f"\nSample filtered events:")
events_in_alarm_windows_df.head(10)

Total alarm windows: 609

First 5 alarm windows:
  AlarmStart_rounded_minutes AlarmEnd_rounded_minutes        window_start  \
0        2022-01-05 08:53:00      2022-01-05 09:33:00 2022-01-05 07:27:00   
1        2022-01-07 09:55:00      2022-01-07 10:00:00 2022-01-07 08:29:00   
2        2022-01-07 13:33:00      2022-01-07 13:36:00 2022-01-07 12:07:00   
3        2022-01-07 14:17:00      2022-01-07 14:19:00 2022-01-07 12:51:00   
4        2022-01-07 14:54:00      2022-01-07 14:58:00 2022-01-07 13:28:00   

           window_end  
0 2022-01-05 10:33:00  
1 2022-01-07 11:00:00  
2 2022-01-07 14:36:00  
3 2022-01-07 15:19:00  
4 2022-01-07 15:58:00  

--- Events Filtering Summary ---
Total events in combined_pv_events_df: 1600520
Events within alarm windows: 7958
Unique alarm episodes with events: 206

ConditionName distribution in filtered events:
ConditionName
CHANGE    7958
Name: count, dtype: int64

Sample filtered events:


,Action,Actor,AreaName,AlarmLimit,Block,Category,ConditionName,Description,EventID,Flags,...,Value,VT_Start,H,TagID,AlarmStatus,AlarmEpisodeIndex,AlarmStart_rounded_minutes,AlarmEnd_rounded_minutes,window_start,window_end
159,NaN,NaN,1L,NaN,NaN,7,CHANGE,MODE,35633314,NaN,...,MAN,2022-01-05 08:41:15.253100,2022_01_05_10,03PIC_1013,NaN,0,2022-01-05 08:53:00,2022-01-05 09:33:00,2022-01-05 07:27:00,2022-01-05 10:33:00
160,NaN,NaN,1L,NaN,NaN,7,CHANGE,MODE,35633314,NaN,...,MAN,2022-01-05 08:41:15.253100,2022_01_05_10,NaN,NaN,0,2022-01-05 08:53:00,2022-01-05 09:33:00,2022-01-05 07:27:00,2022-01-05 10:33:00
199,NaN,NaN,1E,NaN,NaN,7,CHANGE,SP,35633572,NaN,...,20.0000,2022-01-05 08:42:17.358300,2022_01_05_10,NaN,NaN,0,2022-01-05 08:53:00,2022-01-05 09:33:00,2022-01-05 07:27:00,2022-01-05 10:33:00
200,NaN,NaN,1E,NaN,NaN,7,CHANGE,SP,35633572,NaN,...,20.0000,2022-01-05 08:42:17.358300,2022_01_05_10,03LIC_1016,NaN,0,2022-01-05 08:53:00,2022-01-05 09:33:00,2022-01-05 07:27:00,2022-01-05 10:33:00
207,NaN,NaN,1L,NaN,NaN,7,CHANGE,OP,35633602,NaN,...,78.3480,2022-01-05 08:42:23.715200,2022_01_05_10,03PIC_1013,NaN,0,2022-01-05 08:53:00,2022-01-05 09:33:00,2022-01-05 07:27:00,2022-01-05 10:33:00
208,NaN,NaN,1L,NaN,NaN,7,CHANGE,OP,35633602,NaN,...,78.3480,2022-01-05 08:42:23.715200,2022_01_05_10,NaN,NaN,0,2022-01-05 08:53:00,2022-01-05 09:33:00,2022-01-05 07:27:00,2022-01-05 10:33:00
210,NaN,NaN,1L,NaN,NaN,7,CHANGE,OP,35633612,NaN,...,76.3480,2022-01-05 08:42:24.781600,2022_01_05_10,NaN,NaN,0,2022-01-05 08:53:00,2022-01-05 09:33:00,2022-01-05 07:27:00,2022-01-05 10:33:00
212,NaN,NaN,1L,NaN,NaN,7,CHANGE,OP,35633612,NaN,...,76.3480,2022-01-05 08:42:24.781600,2022_01_05_10,03PIC_1013,NaN,0,2022-01-05 08:53:00,2022-01-05 09:33:00,2022-01-05 07:27:00,2022-01-05 10:33:00
213,NaN,NaN,1F,NaN,NaN,7,CHANGE,SP,35633626,NaN,...,20.0000,2022-01-05 08:42:28.203100,2022_01_05_10,NaN,NaN,0,2022-01-05 08:53:00,2022-01-05 09:33:00,2022-01-05 07:27:00,2022-01-05 10:33:00
214,NaN,NaN,1F,NaN,NaN,7,CHANGE,SP,35633626,NaN,...,20.0000,2022-01-05 08:42:28.203100,2022_01_05_10,03LIC_1071,NaN,0,2022-01-05 08:53:00,2022-01-05 09:33:00,2022-01-05 07:27:00,2022-01-05 10:33:00


### Analysis: Episodes with actions only on 1071, 1016, and 1013 tags

We want to find alarm episodes where the only operator actions (CHANGE events) during the action window are on these three specific tags:
- `03LIC_1071` - Target level controller
- `03LIC_1016` - Related level controller  
- `03PIC_1013` - Pressure controller (highly operated)

In [153]:
# Step 1: Get all CHANGE events within alarm windows (using existing alarm_windows)
# Tags we are interested in
TARGET_TAGS = ['03LIC_1071', '03LIC_1016', '03PIC_1013']

# Get all CHANGE events
all_change_events = combined_pv_events_df[combined_pv_events_df['ConditionName'] == 'CHANGE'].copy()

print(f"Total CHANGE events: {len(all_change_events)}")
print(f"Unique sources with CHANGE events: {all_change_events['Source'].nunique()}")

# Find events within each alarm window
event_times = all_change_events['VT_Start'].values
window_starts_arr = alarm_windows['window_start'].values
window_ends_arr = alarm_windows['window_end'].values

# Track which episode each event belongs to
event_episode_idx = np.full(len(event_times), -1, dtype=int)

chunk_size = 100000
for i in range(0, len(event_times), chunk_size):
    chunk = event_times[i:i+chunk_size]
    in_range = (chunk[:, None] >= window_starts_arr) & (chunk[:, None] <= window_ends_arr)
    for j, row_matches in enumerate(in_range):
        if row_matches.any():
            event_episode_idx[i+j] = np.where(row_matches)[0][0]

# Add episode index to events
all_change_events['EpisodeIndex'] = event_episode_idx

# Filter to events within windows
change_events_in_windows = all_change_events[all_change_events['EpisodeIndex'] >= 0].copy()
print(f"\nCHANGE events within alarm windows: {len(change_events_in_windows)}")
print(f"Unique episodes with any CHANGE events: {change_events_in_windows['EpisodeIndex'].nunique()}")

Total CHANGE events: 231714
Unique sources with CHANGE events: 245

CHANGE events within alarm windows: 34908
Unique episodes with any CHANGE events: 488


In [154]:
# Step 2: For each episode, find which have ONLY actions on our target tags (1071, 1016, 1013)
# An episode qualifies if:
# 1. It has at least one CHANGE event on any of the target tags
# 2. ALL CHANGE events in that episode are on the target tags (no other tags)

# Group by episode and get all unique sources (tags) that were operated on
episode_sources = change_events_in_windows.groupby('EpisodeIndex')['Source'].apply(set).reset_index()
episode_sources.columns = ['EpisodeIndex', 'OperatedTags']

# Check if an episode has any target tag action
episode_sources['has_target_tag_action'] = episode_sources['OperatedTags'].apply(
    lambda tags: len(tags.intersection(set(TARGET_TAGS))) > 0
)

# Check if ALL actions are on target tags only
episode_sources['only_target_tags'] = episode_sources['OperatedTags'].apply(
    lambda tags: tags.issubset(set(TARGET_TAGS))
)

# Episodes with at least one target tag action
episodes_with_target_action = episode_sources[episode_sources['has_target_tag_action']]

# Episodes where ONLY target tags were operated on
episodes_only_target_tags = episode_sources[episode_sources['has_target_tag_action'] & episode_sources['only_target_tags']]

print(f"--- Episode Analysis ---")
print(f"Total unique alarm episodes (from SSD): {len(alarm_windows)}")
print(f"Episodes with any CHANGE events: {len(episode_sources)}")
print(f"Episodes with at least one action on target tags (1071/1016/1013): {len(episodes_with_target_action)}")
print(f"Episodes with ONLY actions on target tags: {len(episodes_only_target_tags)}")

# Show breakdown of which target tags were operated
print(f"\n--- Target Tags Breakdown (for episodes with ONLY target tag actions) ---")
for idx, row in episodes_only_target_tags.iterrows():
    tags_str = ', '.join(sorted(row['OperatedTags']))
    # Count how many times each pattern appears
episode_tag_patterns = episodes_only_target_tags['OperatedTags'].apply(lambda x: tuple(sorted(x))).value_counts()
print(episode_tag_patterns)

--- Episode Analysis ---
Total unique alarm episodes (from SSD): 609
Episodes with any CHANGE events: 488
Episodes with at least one action on target tags (1071/1016/1013): 206
Episodes with ONLY actions on target tags: 33

--- Target Tags Breakdown (for episodes with ONLY target tag actions) ---
OperatedTags
(03PIC_1013,)               14
(03LIC_1071,)               13
(03LIC_1016, 03LIC_1071)     5
(03LIC_1071, 03PIC_1013)     1
Name: count, dtype: int64


In [155]:
# Step 3: Get details of the filtered episodes (those with any target tag action)
# We'll plot episodes that have at least one action on 1071/1016/1013 (there could be others as well)

filtered_episode_indices = episodes_with_target_action['EpisodeIndex'].values

# Create episodes dataframe with all details
filtered_episodes = alarm_windows.iloc[filtered_episode_indices].copy()
filtered_episodes['EpisodeIndex'] = filtered_episode_indices
filtered_episodes['EpisodeID'] = range(1, len(filtered_episodes) + 1)

# Rename columns to match expected format
filtered_episodes = filtered_episodes.rename(columns={
    'earliest_transition_start': 'EarliestTransitionStart'
})

# Calculate durations
filtered_episodes['AlarmDurationMinutes'] = (
    (filtered_episodes['AlarmEnd_rounded_minutes'] - filtered_episodes['AlarmStart_rounded_minutes'])
    .dt.total_seconds() / 60
)
filtered_episodes['TransitionToAlarmMinutes'] = (
    (filtered_episodes['AlarmStart_rounded_minutes'] - filtered_episodes['EarliestTransitionStart'])
    .dt.total_seconds() / 60
)
filtered_episodes['TotalWindowMinutes'] = (
    (filtered_episodes['window_end'] - filtered_episodes['window_start'])
    .dt.total_seconds() / 60
)

# Add info about which tags were operated in each episode
filtered_episodes = filtered_episodes.merge(
    episodes_with_target_action[['EpisodeIndex', 'OperatedTags']],
    on='EpisodeIndex',
    how='left'
)
filtered_episodes['OperatedTagsList'] = filtered_episodes['OperatedTags'].apply(lambda x: ', '.join(sorted(x)))

print(f"--- Filtered Episodes Summary ---")
print(f"Total episodes to analyze: {len(filtered_episodes)}")
print(f"\nDuration Statistics (minutes):")
print(f"  Alarm Duration - Mean: {filtered_episodes['AlarmDurationMinutes'].mean():.1f}, "
      f"Median: {filtered_episodes['AlarmDurationMinutes'].median():.1f}, "
      f"Min: {filtered_episodes['AlarmDurationMinutes'].min():.1f}, "
      f"Max: {filtered_episodes['AlarmDurationMinutes'].max():.1f}")
print(f"  Transition to Alarm - Mean: {filtered_episodes['TransitionToAlarmMinutes'].mean():.1f}, "
      f"Median: {filtered_episodes['TransitionToAlarmMinutes'].median():.1f}")
print(f"\nDate Range:")
print(f"  First episode: {filtered_episodes['AlarmStart_rounded_minutes'].min()}")
print(f"  Last episode: {filtered_episodes['AlarmStart_rounded_minutes'].max()}")

# Show first few episodes
print(f"\nFirst 10 filtered episodes:")
filtered_episodes[['EpisodeID', 'AlarmStart_rounded_minutes', 'AlarmEnd_rounded_minutes', 
                   'AlarmDurationMinutes', 'OperatedTagsList']].head(10)

--- Filtered Episodes Summary ---
Total episodes to analyze: 206

Duration Statistics (minutes):
  Alarm Duration - Mean: 36.8, Median: 10.5, Min: 0.0, Max: 290.0
  Transition to Alarm - Mean: 84.2, Median: 86.0

Date Range:
  First episode: 2022-01-05 08:53:00
  Last episode: 2025-06-22 17:14:00

First 10 filtered episodes:


,EpisodeID,AlarmStart_rounded_minutes,AlarmEnd_rounded_minutes,AlarmDurationMinutes,OperatedTagsList
0,1,2022-01-05 08:53:00,2022-01-05 09:33:00,40.0,"03FIC_1085, 03FIC_3435, 03HIC_1141, 03HIC_1151..."
1,2,2022-01-07 14:17:00,2022-01-07 14:19:00,2.0,"03FIC_3435, 03LIC_1016"
2,3,2022-01-23 14:42:00,2022-01-23 15:39:00,57.0,"03FIC_3435, 03GHS_0152AA, 03GM_0152A, 03HIC_11..."
3,4,2022-01-23 17:02:00,2022-01-23 17:13:00,11.0,"03FIC_3435, 03GHS_0152AA, 03GM_0152A, 03HIC_31..."
4,5,2022-01-23 23:32:00,2022-01-23 23:56:00,24.0,"03EM_1511A, 03EM_1511B, 03FIC_3415, 03FIC_3435..."
5,6,2022-02-09 19:12:00,2022-02-09 19:45:00,33.0,"03EM_1511A, 03FIC_3415, 03FIC_3435, 03GM_0151B..."
6,7,2022-02-10 15:21:00,2022-02-10 15:26:00,5.0,"03FIC_3435, 03LIC_1071, 03LIC_1097"
7,8,2022-03-04 11:41:00,2022-03-04 11:50:00,9.0,"03FIC_3415, 03FIC_3435, 03GM_0152A, 03GM_0152B..."
8,9,2022-03-25 23:04:00,2022-03-25 23:10:00,6.0,"03FIC_3415, 03FIC_3435, 03HIC_1141, 03HIC_1151..."
9,10,2022-03-28 08:48:00,2022-03-28 09:06:00,18.0,"03FIC_3435, 03HIC_1141, 03HIC_1151, 03HIC_3100..."


In [156]:
# Step 4: Create visualization function using generate_episode_plots.py template
import importlib.util
from pathlib import Path

# Load generate_episode_plots module directly from script path
_gep_path = Path('.skills/episode-analyzer/scripts/generate_episode_plots.py')
_spec = importlib.util.spec_from_file_location('generate_episode_plots', _gep_path)
_gep = importlib.util.module_from_spec(_spec)
_spec.loader.exec_module(_gep)

# Read config-driven operator action settings
_action_mode = _gep.CONFIG.get('visualization', {}).get('operator_action_mode', 'list')
_max_tags = _gep.CONFIG.get('visualization', {}).get('max_operator_action_tags', 20)

# Load operating limits (used by generate_episode_plots)
limits_df = _gep.load_operating_limits('DATA/operating_limits.csv')

def create_episode_visualization(episode, ts_df, events_df, context_minutes=30):
    """
    Create comprehensive visualization for a single episode using the same
    template as generate_episode_plots.py (all tags + OP/PV grouping).
    """
    return _gep.create_episode_plot(
        episode=episode,
        ts_df=ts_df,
        events_df=events_df,
        ssd_df=ssd_df,
        limits_df=limits_df,
        context_minutes=context_minutes
    )

print("Visualization function created successfully (generate_episode_plots template)!")
print(f"Operator action mode: {_action_mode} | Max tags: {_max_tags}")

Visualization function created successfully (generate_episode_plots template)!
Operator action mode: all | Max tags: 200


In [157]:
# Step 6: Detailed statistics on the filtered episodes

print("=" * 70)
print("DETAILED STATISTICS: Episodes with Target Tag Actions (1071/1016/1013)")
print("=" * 70)

# Overall counts
print(f"\n📊 OVERALL COUNTS")
print(f"   Total SSD alarm episodes: {len(alarm_windows)}")
print(f"   Episodes with any CHANGE events: {len(episode_sources)}")
print(f"   Episodes with target tag actions: {len(filtered_episodes)} ({100*len(filtered_episodes)/len(alarm_windows):.1f}%)")

# Tag combination analysis
print(f"\n📊 TAG COMBINATION PATTERNS")
tag_patterns = filtered_episodes['OperatedTagsList'].value_counts()
print(f"   Unique patterns: {len(tag_patterns)}")
print(f"\n   Top 15 patterns:")
for pattern, count in tag_patterns.head(15).items():
    print(f"   {count:4d} episodes - {pattern}")

# Actions per episode
print(f"\n📊 ACTIONS PER EPISODE")
actions_per_episode = change_events_in_windows[
    change_events_in_windows['EpisodeIndex'].isin(filtered_episode_indices)
].groupby('EpisodeIndex').size()
print(f"   Mean actions per episode: {actions_per_episode.mean():.1f}")
print(f"   Median actions per episode: {actions_per_episode.median():.1f}")
print(f"   Min: {actions_per_episode.min()}, Max: {actions_per_episode.max()}")

# Actions by tag
print(f"\n📊 ACTIONS BY TAG (within filtered episodes)")
target_tag_actions = change_events_in_windows[
    (change_events_in_windows['EpisodeIndex'].isin(filtered_episode_indices)) &
    (change_events_in_windows['Source'].isin(TARGET_TAGS))
]
print(target_tag_actions['Source'].value_counts().to_string())

# Duration analysis
print(f"\n📊 ALARM DURATION ANALYSIS (filtered episodes)")
print(f"   Mean: {filtered_episodes['AlarmDurationMinutes'].mean():.1f} min")
print(f"   Median: {filtered_episodes['AlarmDurationMinutes'].median():.1f} min")
print(f"   Std Dev: {filtered_episodes['AlarmDurationMinutes'].std():.1f} min")
print(f"   Min: {filtered_episodes['AlarmDurationMinutes'].min():.1f} min")
print(f"   Max: {filtered_episodes['AlarmDurationMinutes'].max():.1f} min")
print(f"   Q25: {filtered_episodes['AlarmDurationMinutes'].quantile(0.25):.1f} min")
print(f"   Q75: {filtered_episodes['AlarmDurationMinutes'].quantile(0.75):.1f} min")

# Time distribution
print(f"\n📊 TIME DISTRIBUTION BY YEAR")
filtered_episodes['Year'] = filtered_episodes['AlarmStart_rounded_minutes'].dt.year
year_dist = filtered_episodes.groupby('Year').agg({
    'EpisodeID': 'count',
    'AlarmDurationMinutes': 'mean'
}).rename(columns={'EpisodeID': 'Count', 'AlarmDurationMinutes': 'AvgDuration'})
print(year_dist.to_string())

# Episodes with ONLY target tags (no other tags)
only_target = filtered_episodes[
    filtered_episodes['OperatedTags'].apply(lambda x: x.issubset(set(TARGET_TAGS)))
]
print(f"\n📊 EPISODES WITH ONLY TARGET TAGS (no other tags operated)")
print(f"   Count: {len(only_target)} ({100*len(only_target)/len(filtered_episodes):.1f}% of filtered)")
if len(only_target) > 0:
    print(f"   Patterns:")
    print(only_target['OperatedTagsList'].value_counts().head(10).to_string())

DETAILED STATISTICS: Episodes with Target Tag Actions (1071/1016/1013)

📊 OVERALL COUNTS
   Total SSD alarm episodes: 609
   Episodes with any CHANGE events: 488
   Episodes with target tag actions: 206 (33.8%)

📊 TAG COMBINATION PATTERNS
   Unique patterns: 117

   Top 15 patterns:
     16 episodes - 03FIC_3435, 03PIC_1013
     14 episodes - 03PIC_1013
     13 episodes - 03FIC_3435, 03HIC_1151, 03PIC_1013
     13 episodes - 03LIC_1071
      9 episodes - 03PIC_1013, 03TIC_1009
      6 episodes - 03LIC_1034, 03PIC_1013
      5 episodes - 03LIC_1016, 03LIC_1071
      5 episodes - 03FIC_3435, 03PIC_1013, 03TIC_1009
      4 episodes - 03HIC_1151, 03PIC_1013
      4 episodes - 03HIC_1141, 03HIC_1151, 03LIC_1071, 03PIC_1013
      3 episodes - 03FIC_3435, 03LIC_1034, 03PIC_1013
      3 episodes - 03FIC_3435, 03LIC_1071
      3 episodes - 03FIC_3435, 03HIC_1141, 03HIC_1151, 03HIC_3100, 03HIC_3132, 03PIC_1013
      2 episodes - 03LIC_1034, 03LIC_1085, 03PIC_1013
      2 episodes - 03FIC_3435, 0

### Export Episode Data to Excel and Generate All Graphs

Create an Excel file with:
- Episode info (start, end, duration)
- Tags operated on (from CHANGE events)
- Tags that deviated (from SSD data)

Also save all episode visualizations to RESULTS/episode_target_tags_plots/

In [158]:
# Step 7: Create Excel file with episode details (operated tags + deviated tags) for ALL episodes
import os
from pathlib import Path

# Define target tags for flags
TARGET_TAGS = ['03LIC_1071', '03LIC_1016', '03PIC_1013']

# Create output directory for plots
output_dir = Path('RESULTS/episode_all_operator_action_plots')
output_dir.mkdir(parents=True, exist_ok=True)

# Get deviated tags for each episode from ssd_df
# ssd_df has TagName column and AlarmStart/AlarmEnd to identify episodes
def get_deviated_tags_for_episode(alarm_start, alarm_end, ssd_df):
    """Get tags that deviated (out of steady state) during an episode."""
    mask = (ssd_df['AlarmStart_rounded_minutes'] == alarm_start) & \
           (ssd_df['AlarmEnd_rounded_minutes'] == alarm_end)
    return set(ssd_df[mask]['TagName'].unique())

# Build ALL episodes from alarm_windows
episodes_all = alarm_windows.reset_index(drop=True).copy()
episodes_all['EpisodeIndex'] = episodes_all.index
episodes_all['EpisodeID'] = episodes_all.index + 1
episodes_all = episodes_all.rename(columns={'earliest_transition_start': 'EarliestTransitionStart'})

# Build operated tags per episode by directly querying events for each episode's window
# This correctly handles overlapping windows (where events can belong to multiple episodes)
all_change_events = combined_pv_events_df[combined_pv_events_df['ConditionName'] == 'CHANGE'].copy()

def get_operated_tags_for_episode(window_start, window_end, events_df):
    """Get tags that had CHANGE events within the episode window."""
    mask = (events_df['VT_Start'] >= window_start) & (events_df['VT_Start'] <= window_end)
    return set(events_df[mask]['Source'].unique())

# Apply to each episode
episodes_all['OperatedTags'] = episodes_all.apply(
    lambda row: get_operated_tags_for_episode(row['window_start'], row['window_end'], all_change_events),
    axis=1
)
episodes_all['HasOperatorActions'] = episodes_all['OperatedTags'].apply(lambda x: len(x) > 0)

# Calculate durations
episodes_all['AlarmDurationMinutes'] = (
    (episodes_all['AlarmEnd_rounded_minutes'] - episodes_all['AlarmStart_rounded_minutes'])
    .dt.total_seconds() / 60
    )
episodes_all['TotalWindowMinutes'] = (
    (episodes_all['window_end'] - episodes_all['window_start']).dt.total_seconds() / 60
    )

# Build the comprehensive episode dataframe
episode_data = []
target_set = set(TARGET_TAGS)

for idx, ep in episodes_all.iterrows():
    alarm_start = ep['AlarmStart_rounded_minutes']
    alarm_end = ep['AlarmEnd_rounded_minutes']
    
    # Get deviated tags from SSD
    deviated_tags = get_deviated_tags_for_episode(alarm_start, alarm_end, ssd_df)
    operated_tags = ep['OperatedTags']
    has_only_target = operated_tags.issubset(target_set) if operated_tags else False
    
    episode_data.append({
        'EpisodeID': ep['EpisodeID'],
        'AlarmStart': alarm_start,
        'AlarmEnd': alarm_end,
        'AlarmDurationMinutes': ep['AlarmDurationMinutes'],
        'TotalWindowMinutes': ep['TotalWindowMinutes'],
        'OperatedTags': ', '.join(sorted(operated_tags)),
        'OperatedTagsCount': len(operated_tags),
        'DeviatedTags': ', '.join(sorted(deviated_tags)),
        'DeviatedTagsCount': len(deviated_tags),
        'HasOperatorActions': ep['HasOperatorActions'],
        'HasOnlyTargetTags': has_only_target,
        'Has1071Action': '03LIC_1071' in operated_tags,
        'Has1016Action': '03LIC_1016' in operated_tags,
        'Has1013Action': '03PIC_1013' in operated_tags
    })

episode_export_df = pd.DataFrame(episode_data)

# Save to Excel
excel_path = 'RESULTS/episode_all_operator_action_plots/episodes_all_with_actions_and_deviations.xlsx'
episode_export_df.to_excel(excel_path, index=False)

print(f"✅ Saved episode data to: {excel_path}")
print(f"   Total episodes: {len(episode_export_df)}")
print(f"   Episodes with operator actions: {episode_export_df['HasOperatorActions'].sum()}")
print(f"\nColumn summary:")
print(episode_export_df[['EpisodeID', 'AlarmDurationMinutes', 'TotalWindowMinutes', 'OperatedTagsCount', 
                         'DeviatedTagsCount', 'HasOperatorActions']].describe())

✅ Saved episode data to: RESULTS/episode_all_operator_action_plots/episodes_all_with_actions_and_deviations.xlsx
   Total episodes: 609
   Episodes with operator actions: 538

Column summary:
        EpisodeID  AlarmDurationMinutes  TotalWindowMinutes  \
count  609.000000            609.000000          609.000000   
mean   305.000000             22.811166          167.208539   
std    175.947435             42.409681           42.552029   
min      1.000000              0.000000           99.000000   
25%    153.000000              4.000000          149.000000   
50%    305.000000              6.000000          152.000000   
75%    457.000000             22.000000          168.000000   
max    609.000000            356.000000          502.000000   

       OperatedTagsCount  DeviatedTagsCount  
count         609.000000         609.000000  
mean            3.643678          15.990148  
std             4.538157           3.507028  
min             0.000000           4.000000  
25%       

In [159]:
# Step 8: Generate and save episode visualizations for all episodes WITH operator actions

PLOT_CONTEXT_MINUTES = 60  # matches AlarmEnd + 60 min window
episodes_with_actions = episodes_all[episodes_all['HasOperatorActions']].copy()
episodes_with_actions['OperatedTagsList'] = episodes_with_actions['OperatedTags'].apply(lambda x: ', '.join(sorted(x)))

print(f"Generating and saving visualizations for all {len(episodes_with_actions)} episodes with operator actions...")
print(f"Output directory: {output_dir}")
print(f"Context minutes: {PLOT_CONTEXT_MINUTES}")
print(f"{'='*70}")

saved_count = 0
error_count = 0

for idx, (_, episode) in enumerate(episodes_with_actions.iterrows()):
    try:
        episode_id = episode['EpisodeID']
        
        # Create visualization
        fig = create_episode_visualization(
            episode,
            op_pv_data_df,
            combined_pv_events_df,
            context_minutes=PLOT_CONTEXT_MINUTES
        )
        
        # Save as HTML
        filename = f"episode_{episode_id:04d}.html"
        filepath = output_dir / filename
        fig.write_html(filepath, include_plotlyjs='cdn')
        
        saved_count += 1
        
        # Progress update every 20 episodes
        if (idx + 1) % 20 == 0:
            print(f"   Progress: {idx + 1}/{len(episodes_with_actions)} episodes saved...")
            
    except Exception as e:
        error_count += 1
        print(f"   ⚠️ Error saving episode {episode_id}: {str(e)}")

print(f"\n{'='*70}")
print(f"✅ Saved {saved_count} episode visualizations to: {output_dir}/")
if error_count > 0:
    print(f"⚠️ Errors: {error_count}")

Generating and saving visualizations for all 538 episodes with operator actions...
Output directory: RESULTS/episode_all_operator_action_plots
Context minutes: 60
   Progress: 20/538 episodes saved...
   Progress: 40/538 episodes saved...
   Progress: 60/538 episodes saved...
   Progress: 80/538 episodes saved...
   Progress: 100/538 episodes saved...
   Progress: 120/538 episodes saved...
   Progress: 140/538 episodes saved...
   Progress: 160/538 episodes saved...
   Progress: 180/538 episodes saved...
   Progress: 200/538 episodes saved...
   Progress: 220/538 episodes saved...
   Progress: 240/538 episodes saved...
   Progress: 260/538 episodes saved...
   Progress: 280/538 episodes saved...
   Progress: 300/538 episodes saved...
   Progress: 320/538 episodes saved...
   Progress: 340/538 episodes saved...
   Progress: 360/538 episodes saved...
   Progress: 380/538 episodes saved...
   Progress: 400/538 episodes saved...
   Progress: 420/538 episodes saved...
   Progress: 440/538 e